# Documentation and Logic
Yahoo finance does not explicitly forbid web scraping.  They do offer an API, but according to the documentation the historical data was limited to hard values like 1 month, 1 day, ect.  You could not specify dates.  There is also a yahoo finance web scrapping package for python, but I found that it failed to get a lot of the data.  Random dates would come up as NA even though the data existed.  I’m making a de novo scraping to get around that issue.  We are primarily interested in historical data, so the code focuses on fetching the historical data table for the specified entities.

In [1]:
import pandas as pd
import time

from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager

# Set the executable path and initialize Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless = True)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\karar\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


In [2]:
market_tickers = [
    '%5EGSPC', # S&P 500
    '%5EIXIC' # NASDAQ Composite
]

# cryptcurrency tickers
crypto_tickers = [
#     'AAVE',
#     'BNB',
    'BTC',
#     'ADA',
#     'LINK',
#     'ATOM',
#     'CRO',
#     'DOGE',
#     'EOS',
#     'ETH',
#     'MIOTA',
#     'LTC',
#     'XMR',
#     'XEM',
#     'DOT',
#     'SOL',
#     'XLM',
#     'USDT',
#     'TRX',
#     'UNI',
#     'USDC',
#     'WBTC',
#     'XRP'
]

crypto_tickers = [f"{tick}-USD" for tick in crypto_tickers]

In [3]:
def get_historical_data(browser, ticker, start, end):
    """Given a list of crypto tickers and a start and end data (format = ‘mm/dd/yyyy’) return the historical price data from Yahoo Finance."""
    
    final_df = pd.DataFrame()
    
    # get the base URL and UNIX date ranges
    base_url = 'https://finance.yahoo.com/quote/'
    start = create_epoch(start) - 86400
    end = create_epoch(end)
    days_100 = 86400 * 100
    
    # get info in chunks of 100 because of how YF loads
    while start < end:
        
        # get 100 days past the start as the end
        current_end = end if start + days_100 > end else start + days_100
        
        # visit website and prase out data
        url = f"{base_url}{ticker}/history?period1={start}&period2={current_end}"
        browser.visit(url)
        html = browser.html
        df = parse_html(bs(html, 'html.parser'))
        
        # add data to final df and make the current end_date next start date 
        final_df = final_df.append(df)
        start = current_end
        
    # final df cleaning
    final_df['Date'] = pd.to_datetime(final_df['Date'])
    final_df = final_df.sort_values(by = 'Date')
    
    return final_df.reset_index(drop = True) 

In [4]:
def create_epoch(date):
    """Given data (format = ‘mm/dd/yyyy’) return the Epoch."""
    date_time = f"{date} 12:59:59"
    pattern = '%m/%d/%Y %H:%M:%S'
    return int(time.mktime(time.strptime(date_time, pattern)))

In [5]:
# This is the function that will have to change if Yahoo Finance changes their website
def parse_html(fin_soup):
    """Given html data from Yahoo Finance return a price history dataframe."""
    
    # Parse out the rows
    rows = []
    for section in fin_soup.find('table').children:
        for tr in section:
            row = []
            for tx in tr:
                row.append(tx.text)
            rows.append(row)

    # make dataframe
    return pd.DataFrame(rows[1:len(rows) - 1], columns = rows[0])

In [6]:
history = []
# for tick in market_tickers:
#     history.append(get_historical_data(browser, ticker, '10/20/2015', '10/30/2015'))
for tick in crypto_tickers:
    history.append(get_historical_data(browser, tick, '12/1/2017', '12/1/2018'))
browser.quit()
history[0].shape

(367, 7)

In [7]:
history[0].head(50)

,Date,Open,High,Low,Close*,Adj Close**,Volume
0,2017-12-01,"10,198.60","11,046.70","9,694.65","10,975.60","10,975.60","6,783,119,872"
1,2017-12-02,"10,978.30","11,320.20","10,905.10","11,074.60","11,074.60","5,138,500,096"
2,2017-12-03,"11,082.70","11,858.70","10,862.00","11,323.20","11,323.20","6,608,309,760"
3,2017-12-04,"11,315.40","11,657.20","11,081.80","11,657.20","11,657.20","6,132,409,856"
4,2017-12-05,"11,685.70","12,032.00","11,604.60","11,916.70","11,916.70","6,895,260,160"
5,2017-12-06,"11,923.40","14,369.10","11,923.40","14,291.50","14,291.50","12,656,300,032"
6,2017-12-07,"14,266.10","17,899.70","14,057.30","17,899.70","17,899.70","17,950,699,520"
7,2017-12-08,"17,802.90","18,353.40","14,336.90","16,569.40","16,569.40","21,135,998,976"
8,2017-12-09,"16,523.30","16,783.00","13,674.90","15,178.20","15,178.20","13,911,300,096"
9,2017-12-10,"15,168.40","15,850.60","13,226.60","15,455.40","15,455.40","13,433,299,968"


In [8]:
history[0].tail(20)

,Date,Open,High,Low,Close*,Adj Close**,Volume
347,2018-11-12,"6,411.76","6,434.21","6,360.47","6,371.27","6,371.27","4,295,770,000"
348,2018-11-13,"6,373.19","6,395.27","6,342.67","6,359.49","6,359.49","4,503,800,000"
349,2018-11-14,"6,351.24","6,371.55","5,544.09","5,738.35","5,738.35","7,398,940,000"
350,2018-11-15,"5,736.15","5,774.82","5,358.38","5,648.03","5,648.03","7,032,140,000"
351,2018-11-16,"5,645.32","5,657.02","5,498.94","5,575.55","5,575.55","5,279,320,000"
352,2018-11-17,"5,578.58","5,578.58","5,519.56","5,554.33","5,554.33","4,303,150,000"
353,2018-11-18,"5,559.74","5,653.61","5,559.74","5,623.54","5,623.54","4,159,680,000"
354,2018-11-19,"5,620.78","5,620.78","4,842.91","4,871.49","4,871.49","7,039,560,000"
355,2018-11-20,"4,863.93","4,951.61","4,272.11","4,451.87","4,451.87","8,428,290,000"
356,2018-11-21,"4,465.54","4,675.73","4,343.98","4,602.17","4,602.17","6,120,120,000"
